In [ ]:
import ccxt
import pandas as pd
from datetime import datetime, timedelta

# Initialize the exchange
exchange = ccxt.binance()  # You can change this to any other supported exchange

def fetch_historical_ohlcv(symbol, start_date, end_date):
    since = exchange.parse8601(start_date.isoformat())
    end_time = exchange.parse8601(end_date.isoformat())
    ohlcv_data = []
    
    while since < end_time:
        # Fetch the data
        data = exchange.fetch_ohlcv(symbol, timeframe='1d', since=since, limit=1000)
        if len(data) == 0:
            break
        ohlcv_data.extend(data)
        since = data[-1][0] + 86400000  # Add one day in milliseconds to avoid overlap
    
    return ohlcv_data

def date_to_timestamp(date_str):
    dt = datetime.strptime(date_str, '%Y-%m-%d')
    return int(dt.timestamp() * 1000)

# Define the cryptocurrencies and the date range
cryptocurrencies = ['BTC/USDT', 'ETH/USDT', 'LTC/USDT', 'XRP/USDT', 'ADA/USDT', 'DOT/USDT']
end_date = datetime.now()
start_date = end_date - timedelta(days=90)

# Fetch and save data for each cryptocurrency
for symbol in cryptocurrencies:
    ohlcv = fetch_historical_ohlcv(symbol, start_date, end_date)
    if not ohlcv:
        print(f"No data fetched for {symbol}")
        continue
    
    # Extract the required fields and convert to DataFrame
    records = []
    for data in ohlcv:
        record = {
            'timestamp': datetime.fromtimestamp(data[0] / 1000),
            'open': data[1],
            'high': data[2],
            'low': data[3],
            'close': data[4],
            'volume': data[5]
        }
        records.append(record)
    
    df = pd.DataFrame(records)
    df.set_index('timestamp', inplace=True)
    # Save to CSV
    csv_filename = symbol.replace("/", "_") + '_ohlcv_data.csv'
    df.to_csv(csv_filename)
    
print("Data fetched and saved successfully!")
